In [2]:
%logstop
%logstart -rtq ~/.logs/dw.py append
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

In [3]:
from static_grader import grader

# DW Miniproject
## Introduction

The objective of this miniproject is to exercise your ability to wrangle tabular data set and aggregate large data sets into meaningful summary statistics. We'll work with the same medical data used in the `pw` miniproject but leverage the power of Pandas to more efficiently represent and act on our data.

## Downloading the data

We first need to download the data we'll be using from Amazon S3:

In [4]:
!mkdir dw-data
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/201701scripts_sample.csv.gz -nc -P ./dw-data/
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/201606scripts_sample.csv.gz -nc -P ./dw-data/
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/practices.csv.gz -nc -P ./dw-data/
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/chem.csv.gz -nc -P ./dw-data/

mkdir: cannot create directory ‘dw-data’: File exists
File ‘./dw-data/201701scripts_sample.csv.gz’ already there; not retrieving.

File ‘./dw-data/201606scripts_sample.csv.gz’ already there; not retrieving.

File ‘./dw-data/practices.csv.gz’ already there; not retrieving.

File ‘./dw-data/chem.csv.gz’ already there; not retrieving.



## Loading the data

Similar to the `PW` miniproject, the first step is to read in the data. The data files are stored as compressed CSV files. You can load the data into a Pandas DataFrame by making use of the `gzip` package to decompress the files and Panda's `read_csv` methods to parse the data into a DataFrame. You may want to check the Pandas documentation for parsing [CSV](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) files for reference.

For a description of the data set please, refer to the [PW miniproject](./pw.ipynb). **Note that all questions make use of the 2017 data only, except for Question 5 which makes use of both the 2017 and 2016 data.**

In [4]:
import pandas as pd
import numpy as np
import gzip

In [5]:
# load the 2017 data
scripts = pd.read_csv('./dw-data/201701scripts_sample.csv.gz', compression='gzip', header=0, sep=',', quotechar='"')
scripts.head()

practice   bnf_code                              bnf_name  items   nic  \
0   N85639  0106020C0                 Bisacodyl_Tab E/C 5mg      1  0.39   
1   N85639  0106040M0      Movicol Plain_Paed Pdr Sach 6.9g      1  4.38   
2   N85639  0301011R0    Salbutamol_Inha 100mcg (200 D) CFF      1  1.50   
3   N85639  0304010G0  Chlorphenamine Mal_Oral Soln 2mg/5ml      1  2.62   
4   N85639  0401020K0                      Diazepam_Tab 2mg      1  0.16   

   act_cost  quantity  
0      0.47        12  
1      4.07        30  
2      1.40         1  
3      2.44       150  
4      0.26         6

In [6]:
col_names=[ 'code', 'name', 'addr_1', 'addr_2', 'borough', 'village', 'post_code']
practices = pd.read_csv('./dw-data/practices.csv.gz', compression='gzip', names = col_names, header=0, sep=',', quotechar='"')
practices.head()

code                             name                   addr_1  \
0  A81002       QUEENS PARK MEDICAL CENTRE  QUEENS PARK MEDICAL CTR   
1  A81003        VICTORIA MEDICAL PRACTICE        THE HEALTH CENTRE   
2  A81004           WOODLANDS ROAD SURGERY         6 WOODLANDS ROAD   
3  A81005               SPRINGWOOD SURGERY       SPRINGWOOD SURGERY   
4  A81006  TENNANT STREET MEDICAL PRACTICE     TENNANT ST MED PRACT   

          addr_2           borough    village post_code  
0  FARRER STREET  STOCKTON ON TEES  CLEVELAND  TS18 2AW  
1  VICTORIA ROAD        HARTLEPOOL  CLEVELAND  TS26 8DB  
2            NaN     MIDDLESBROUGH  CLEVELAND   TS1 3BE  
3   RECTORY LANE       GUISBOROUGH        NaN  TS14 7DJ  
4  FARRER STREET  STOCKTON ON TEES  CLEVELAND  TS18 2AT

In [7]:
chem = pd.read_csv('./dw-data/chem.csv.gz', compression='gzip', header=0, sep=',', quotechar='"')
chem.head()

CHEM SUB                                NAME
0  0101010A0                     Alexitol Sodium
1  0101010B0                          Almasilate
2  0101010C0                 Aluminium Hydroxide
3  0101010D0  Aluminium Hydroxide With Magnesium
4  0101010E0                        Hydrotalcite

Now that we've loaded in the data, let's first replicate our results from the `PW` miniproject. Note that we are now working with a larger data set so the answers will be different than in the `PW` miniproject even if the analysis is the same.

## Question 1: summary_statistics

In the `PW` miniproject we first calculated the total, mean, standard deviation, and quartile statistics of the `'items'`, `'quantity'`', `'nic'`, and `'act_cost'` fields. To do this we had to write some functions to calculate the statistics and apply the functions to our data structure. The DataFrame has a `describe` method that will calculate most (not all) of these things for us.

Submit the summary statistics to the grader as a list of tuples: [('act_cost', (total, mean, std, q25, median, q75)), ...]

In [8]:
desc = scripts.describe()
desc

items            nic       act_cost       quantity
count  973193.000000  973193.000000  973193.000000  973193.000000
mean        9.133136      73.058915      67.986613     741.329835
std        29.204198     188.070257     174.401703    3665.426958
min         1.000000       0.000000       0.040000       0.000000
25%         1.000000       7.800000       7.330000      28.000000
50%         2.000000      22.640000      21.220000     100.000000
75%         6.000000      65.000000      60.670000     350.000000
max      2384.000000   16320.000000   15108.320000  577720.000000

In [9]:
#summary_stats = [('items', (0,) * 6), ('quantity', (0,) * 6), ('nic', (0,) * 6), ('act_cost', (0,) * 6)]
def summary(key):
    total = scripts[key].sum()
    avg = scripts[key].describe()[1]
    s = scripts[key].describe()[2]
    q25 = scripts[key].describe()[4]
    med = scripts[key].median()
    q75 = scripts[key].describe()[6]
    return (total, avg, s, q25, med, q75)

summary_stats = [('items', summary('items')), 
                 ('quantity', summary('quantity')), 
                 ('nic', summary('nic')), 
                 ('act_cost', summary('act_cost'))]

In [10]:
grader.score.dw__summary_statistics(summary_stats)

Your score:  1.0


## Question 2: most_common_item

We can also easily compute summary statistics on groups within the data. In the `pw` miniproject we had to explicitly construct the groups based on the values of a particular field. Pandas will handle that for us via the `groupby` method. This process is [detailed in the Pandas documentation](https://pandas.pydata.org/pandas-docs/stable/groupby.html).

Use `groupby` to calculate the total number of items dispensed for each `'bnf_name'`. Find the item with the highest total and return the result as `[(bnf_name, total)]`.

In [11]:
bnf_group = scripts.groupby(['bnf_name'], sort=False)['items'].sum().reset_index()
max_item = bnf_group[bnf_group['items'] == bnf_group['items'].max()]

In [12]:
max_ = (max_item['bnf_name'].to_string().split('61    ')[-1], pd.to_numeric(max_item['items'].to_string().split(' ')[-1]))

In [13]:
most_common_item = [max_]

In [14]:
grader.score.dw__most_common_item(most_common_item)

Your score:  1.0


## Question 3: items_by_region

Now let's find the most common item by post code. The post code information is in the `practices` DataFrame, and we'll need to `merge` it into the `scripts` DataFrame. Pandas provides [extensive documentation](https://pandas.pydata.org/pandas-docs/stable/merging.html) with diagrammed examples on different methods and approaches for joining data. The `merge` method is only one of many possible options.

Return your results as a list of tuples `(post code, item name, amount dispensed as % of total)`. Sort your results ascending alphabetically by post code and take only results from the first 100 post codes.

**NOTE:** Some practices have multiple postal codes associated with them. Use the alphabetically first postal code. Note some postal codes may have multiple `'bnf_name'` with the same prescription rate for the maximum. In this case, take the alphabetically first `'bnf_name'` (as in the PW miniproject).

In [15]:
# Get minimum post from practice with duplicte post_code
minimum_posts = practices.groupby('code').min()
#minimum_posts['code'].head()

In [16]:
# Merge post_code from minimum_posts to scripts df
merged_data = scripts.merge(minimum_posts[['post_code']], left_on='practice', right_on='code')

In [17]:
# get total items prescribed for each bnf_name for each post_code
group_postal_code = merged_data.groupby(['post_code','bnf_name'])['items'].sum()

In [18]:
# for each post_code in group_postal_code df, get the bnf_name with the maximum total prescription
maximum_data_frame = group_postal_code.loc[group_postal_code.groupby('post_code').idxmax()].to_frame()

# for each postal_code get the total of medicine prescribed
sum_data_frame = group_postal_code.groupby('post_code').apply(sum).to_frame()

# get the max/total ratio for each post_code
region_data = (maximum_data_frame / sum_data_frame).to_records()

In [19]:
#items_by_region = [("B11 4BW", "Salbutamol_Inha 100mcg (200 D) CFF", 0.0310589063)] * 100
items_by_region = sorted(list(map(tuple, region_data)))[:100]

In [20]:
grader.score.dw__items_by_region(items_by_region)

Your score:  1.0


## Question 4: script_anomalies

Drug abuse is a source of human and monetary costs in health care. A first step in identifying practitioners that enable drug abuse is to look for practices where commonly abused drugs are prescribed unusually often. Let's try to find practices that prescribe an unusually high amount of opioids. The opioids we'll look for are given in the list below.

In [21]:
opioids = ['morphine', 'oxycodone', 'methadone', 'fentanyl', 'pethidine', 'buprenorphine', 'propoxyphene', 'codeine']

These are generic names for drugs, not brand names. Generic drug names can be found using the `'bnf_code'` field in `scripts` along with the `chem` table.. Use the list of opioids provided above along with these fields to make a new field in the `scripts` data that flags whether the row corresponds with a opioid prescription.

In [22]:
# remove chem duplicates
chem =chem.drop_duplicates(subset='CHEM SUB')

In [23]:
# create a string of piped opioids name
opioids_joined = '|'.join(opioids)
scripts_chem_joined = scripts.merge(chem, left_on='bnf_code', right_on='CHEM SUB', how='left')

In [24]:
# check if prescription contains any opioids, set not available to false and make the search case insensitive
scripts_chem_joined['Opioids'] = scripts_chem_joined.NAME.str.contains(opioids_joined, na=False, case=False)

In [25]:
scripts_chem_joined = scripts_chem_joined.merge(minimum_posts.reset_index(), left_on='practice', right_on='code', how='left')

In [26]:
#scripts_chem_joined.head(2)

Now for each practice calculate the proportion of its prescriptions containing opioids.

**Hint:** Consider the following list: `[0, 1, 1, 0, 0, 0]`. What proportion of the entries are 1s? What is the mean value?

In [27]:
# create group based on practice
groupby_practice = scripts_chem_joined.groupby(['practice'])

In [28]:
# get ration of opiods prescription per practice
opioids_prescription_in_practice = groupby_practice.Opioids.mean()

How do these proportions compare to the overall opioid prescription rate? Subtract off the proportion of all prescriptions that are opioids from each practice's proportion.

In [29]:
# relative opioids prescription per practice
relative_opioids_per_practice = groupby_practice.Opioids.mean() - scripts_chem_joined.Opioids.mean()

Now that we know the difference between each practice's opioid prescription rate and the overall rate, we can identify which practices prescribe opioids at above average or below average rates. However, are the differences from the overall rate important or just random deviations? In other words, are the differences from the overall rate big or small?

To answer this question we have to quantify the difference we would typically expect between a given practice's opioid prescription rate and the overall rate. This quantity is called the **standard error**, and is related to the **standard deviation**, $\sigma$. The standard error in this case is

$$ \frac{\sigma}{\sqrt{n}} $$

where $n$ is the number of prescriptions each practice made. Calculate the standard error for each practice. Then divide `relative_opioids_per_practice` by the standard errors. We'll call the final result `opioid_scores`.

In [30]:
# get the standard error for each practice
standard_error_per_practice = np.std(scripts_chem_joined.Opioids) / np.sqrt(groupby_practice.Opioids.count())
# get the opioids score for each practice
opioid_scores = relative_opioids_per_practice / standard_error_per_practice

In [31]:
#opioid_scores

The quantity we have calculated in `opioid_scores` is called a **z-score**:

$$ \frac{\bar{X} - \mu}{\sqrt{\sigma^2/n}} $$

Here $\bar{X}$ corresponds with the proportion for each practice, $\mu$ corresponds with the proportion across all practices, $\sigma^2$ corresponds with the variance of the proportion across all practices, and $n$ is the number of prescriptions made by each practice. Notice $\bar{X}$ and $n$ will be different for each practice, while $\mu$ and $\sigma$ are determined across all prescriptions, and so are the same for every z-score. The z-score is a useful statistical tool used for hypothesis testing, finding outliers, and comparing data about different types of objects or events.

Now that we've calculated this statistic, take the 100 practices with the largest z-score. Return your result as a list of tuples in the form `(practice_name, z-score, number_of_scripts)`. Sort your tuples by z-score in descending order. Note that some practice codes will correspond with multiple names. In this case, use the first match when sorting names alphabetically.

In [32]:
series_dict = {'z_score': opioid_scores, 'num_scripts': groupby_practice.Opioids.count()}
statistic_pd = pd.DataFrame(series_dict).reset_index()

In [33]:
scripts_chem_joined = scripts_chem_joined.merge(statistic_pd, left_on='code', right_on='practice')

In [34]:
group_by_practice = scripts_chem_joined.groupby('practice_x')

In [35]:
new_df = group_by_practice[['name','z_score','num_scripts']].min()
new_df.set_index('name', inplace=True)

In [36]:
sorted_list = sorted(list(map(tuple, new_df.to_records())), key= lambda tup: tup[1])
sorted_list.reverse()

In [37]:
#unique_practices = ...
#anomalies = [("NATIONAL ENHANCED SERVICE", 11.6958178629, 7)] * 100

In [38]:
anomalies = sorted_list[:100]

In [39]:
#results.head()

In [40]:
grader.score.dw__script_anomalies(anomalies)

Your score:  1.0


## Question 5: script_growth

Another way to identify anomalies is by comparing current data to historical data. In the case of identifying sites of drug abuse, we might compare a practice's current rate of opioid prescription to their rate 5 or 10 years ago. Unless the nature of the practice has changed, the profile of drugs they prescribe should be relatively stable. We might also want to identify trends through time for business reasons, identifying drugs that are gaining market share. That's what we'll do in this question.

We'll load in beneficiary data from 6 months earlier, June 2016, and calculate the percent growth in prescription rate from June 2016 to January 2017 for each `bnf_name`. We'll return the 50 items with largest growth and the 50 items with the largest shrinkage (i.e. negative percent growth) as a list of tuples sorted by growth rate in descending order in the format `(script_name, growth_rate, raw_2016_count)`. You'll notice that many of the 50 fastest growing items have low counts of prescriptions in 2016. Filter out any items that were prescribed less than 50 times.

In [41]:
scripts16 = pd.read_csv('./dw-data/201606scripts_sample.csv.gz', compression='gzip', header=0, sep=',', quotechar='"')

In [42]:
# count the total items per bnf_name for the 2016 scripts and create a new column count16
drugs_16 = scripts16[['bnf_name', 'items']]
drugs_16 = drugs_16.groupby('bnf_name').count().reset_index()
drugs_16.columns = ['bnf_name','count16']

In [43]:
# count the total items per bnf_name for the 2017 scripts and create a new column count17
drugs_17 = scripts[['bnf_name','items']]
drugs_17 = drugs_17.groupby('bnf_name').count().reset_index()
drugs_17.columns = ['bnf_name','count17']

In [44]:
# merge drugs_17 to drugs_16
drugs = drugs_16.merge(drugs_17, on='bnf_name', how='inner')
# get items where prescription is above 50 items
drugs = drugs[drugs['count16'] >= 50]

In [45]:
# calculate the growth and assign to a new column growth in the drugs df
# growth = count17 - count16 / count16
drugs['growth'] = (drugs['count17'] - drugs['count16']) / drugs['count16']
drugs = drugs[['bnf_name', 'growth', 'count16']]

In [46]:
# We'll return the 50 items with largest growth and the 50 items with the largest shrinkage (i.e. negative percent growth)
# sorted by growth descending
drugs.sort_values('growth', ascending=False, inplace=True)
drugs_final = pd.concat([drugs.iloc[:50], drugs.iloc[-50:]], axis=0)

In [47]:
script_growth = drugs_final

In [48]:
grader.score.dw__script_growth(script_growth)

Your score:  1.0


## Question 6: rare_scripts

Does a practice's prescription costs originate from routine care or from reliance on rarely prescribed treatments? Commonplace treatments can carry lower costs than rare treatments because of efficiencies in large-scale production. While some specialist practices can't help but avoid prescribing rare medicines because there are no alternatives, some practices may be prescribing a unnecessary amount of brand-name products when generics are available. Let's identify practices whose costs disproportionately originate from rarely prescribed items.

First we have to identify which `'bnf_code'` are rare. To do this, find the probability $p$ of a prescription having a particular `'bnf_code'` if the `'bnf_code'` was randomly chosen from the unique options in the beneficiary data. We will call a `'bnf_code'` rare if it is prescribed at a rate less than $0.1p$.

In [67]:
#p = ...
#rates = ...
#rare_codes = ...
#scripts['rare'] = ...

In [66]:
p = 1/len(scripts.bnf_code.unique())
rate = scripts.groupby('bnf_code').bnf_code.count()/scripts.groupby('bnf_code').bnf_code.count().sum()
rates = rate[rate < (p*0.1)]

rates_df = pd.DataFrame(rates).rename(columns={'bnf_code': 'rates'}).reset_index()
rates_df['rare_code'] = rates_df['bnf_code']

scripts_merge = scripts.merge(rates_df, on='bnf_code', how='left')
scripts_merge['rare_code'].fillna(False, inplace=True)

def replace(row):
    if row == False:
        return False
    else:
        return True

scripts_merge['rare'] = scripts_merge['rare_code'].apply(replace)
rare_cost = scripts_merge[scripts_merge.rare == True].groupby(['practice','rare']).act_cost.sum().to_dict()
rare_total_cost = sum(rare_cost.values())
total_act_cost = scripts['act_cost'].sum()
overall_rare_cost_prop = rare_total_cost / total_act_cost
overall_rare_cost_prop

act_sum_per_practice = scripts.groupby('practice').act_cost.sum().to_dict()

In [73]:
rare_cost_key = list(rare_cost.keys())
rare_cost_keys = [k[0] for k in rare_cost_key]

rare_cost_prop = {}
for key,value in act_sum_per_practice.items():
    if key in rare_cost_keys:
        rare_cost_prop[key] = rare_cost[tuple([key, True])]/act_sum_per_practice[key]
    else:
        rare_cost_prop[key] = 0


rare_cost_diff = {}
for key,value in rare_cost_prop.items():
    rare_cost_diff[key] = rare_cost_prop[key] - overall_rare_cost_prop


standard_error = np.std(list(rare_cost_diff.values()))

z_score = {}
for key,value in rare_cost_diff.items():
    z_score[key] = rare_cost_diff[key]/standard_error
    

name_dict = practices[['code','name']].groupby('code')['name'].min().to_dict()

rare_script = []
for key,value in z_score.items():
    for keys,values in name_dict.items():
        if keys == key:
            rare_script.append((keys,values,value))

rare_script = sorted(rare_script, key=lambda x: x[2], reverse=True)
rare_scripts = rare_script[:100]
#rare_scripts

Now we will calculate a z-score for each practice based on this proportion.
First take the difference of `rare_cost_prop` and the proportion of costs originating from rare treatments across all practices.

In [74]:
#relative_rare_cost_prop = ...

Now we will estimate the standard errors (i.e. the denominator of the z-score) by simply taking the standard deviation of this difference.

In [75]:
#standard_errors = ...

Finally compute the z-scores. Return the practices with the top 100 z-scores in the form `(post_code, practice_name, z-score)`. Note that some practice codes will correspond with multiple names. In this case, use the first match when sorting names alphabetically.

In [76]:
#rare_scores = ...

In [77]:
#rare_scripts = [("Y03472", "CONSULTANT DIABETES TEAM", 16.2626871247)] * 100

In [78]:
grader.score.dw__rare_scripts(rare_scripts)

Your score:  1.0


*Copyright &copy; 2020 The Data Incubator.  All rights reserved.*